COMP90073 Assignment 2 - Task 1

Name - Ribhav Shridhar

Student ID - 1037144

Referenced from 
**- https://github.com/antoinedelplace/Cyberattack-Detection**

In [ ]:
pip install pyod  # Installing PyOD package 

In [ ]:
import pandas as pd # Importing required packages
import numpy as np
from pyod.models.iforest import IForest
from sklearn import metrics 

Ingesting data from each of the 3 feature sets

In [ ]:
train_Data_Set1 = pd.read_hdf('set1.h5', key='data')  
train_Data_Set1.reset_index(drop=True, inplace=True)
train_Data_Set1.drop('window_id', axis=1, inplace=True)

In [ ]:
train_Data_Set2 = pd.read_hdf('set2.h5', key='data')
train_Data_Set2.reset_index(drop=True, inplace=True)

In [ ]:
train_Data_Set3 = pd.read_csv('training_data_set3.csv')
train_Data_Set3 = train_Data_Set2.drop(['Unnamed: 0'], axis=1)

In [ ]:
test_Data_Set1 = pd.read_hdf('test_set1.h5', key='data')
test_Data_Set1.reset_index(drop=True, inplace=True)
test_Data_Set1.drop('window_id', axis=1, inplace=True)

In [ ]:
test_Data_Set2 = pd.read_hdf('test_set2.h5', key='data')
test_Data_Set2.reset_index(drop=True, inplace=True)

In [ ]:
test_Data_Set3 = pd.read_csv('test_set3.csv')
test_Data_Set3= test_Data_Set3.drop(['Unnamed: 0'], axis=1)
test_Data_Set3.drop(['Duration_max',	'Duration_median', 'BiDirection_Bytes_median', 'SrcToDst_Bytes_median'], inplace = True, axis = 1)

In [ ]:
valid_Data_Set1 = pd.read_hdf('valid_set1.h5', key='data')
valid_Data_Set1.reset_index(drop=True, inplace=True)
valid_Data_Set1.drop('window_id', axis=1, inplace=True)
valid_Set1_Labels = valid_Data_Set1['Label_<lambda>']
valid_Data_Set1.drop(['Label_<lambda>'], inplace=True, axis=1)

In [ ]:
valid_Data_Set2 = pd.read_hdf('valid_set2.h5', key='data')
valid_Data_Set2.reset_index(drop=True, inplace=True)

In [ ]:
valid_Data_Set3 = pd.read_csv('valid_set3.csv')
valid_Data_Set3= valid_Data_Set3.drop(['Unnamed: 0'], axis=1)
valid_Data_Set3.drop(['Duration_max',	'Duration_median', 'BiDirection_Bytes_median', 'SrcToDst_Bytes_median', 'Label_<lambda>'], inplace = True, axis = 1)

In [ ]:
Test_Data = pd.read_csv("test_data.csv", header=None)
Test_Data.columns = ['Date_Flow_Start', 'Duration','Protocol','Src_IP','Src_Port','Direction','Dst_IP','Dst_Port','State','Source_Service','Dest_Service','Total_Packets','BiDirection_Bytes','SrcToDst_Bytes']

Training the model using Isolation Forest Algorithm

In [ ]:
clf = IForest() # Initially using the default parameters; contamination = 0.1
clf.fit(train_Data_Set1)

Calcalation decision scores for the training data, and computing the threshold

In [ ]:
train_decision_scores = clf.decision_scores_
train_decision_scores_sorted=sorted(train_decision_scores, reverse=True)
con_index = round(len(train_Data_Set1) * clf.contamination)
threshold = train_decision_scores_sorted[con_index-1] # Calculating threshold based on decision scores and the contamination score

In [ ]:
valid_decision_scores = clf1.decision_function(valid_Data_Set1)  # Outlier scores for validation data
valid_decision_scores = pd.Series(valid_decision_scores)

In [ ]:
valid_SrcIP = np.load('valid_set1_IPs.npy',allow_pickle=True)

In [ ]:
valid_outliers = []
y_pred_valid = []
for score in range(0,len(valid_decision_scores)):
  if valid_decision_scores[score] > threshold:
    reg = (valid_SrcIP[score], valid_decision_scores[score])
    valid_outliers.append(reg)
    y_pred_valid.append(1.0)
  else:
    y_pred_valid.append(0.0)

In [ ]:
precision = metrics.precision_score(Y_valid1, y_pred_valid)
accuracy = metrics.accuracy_score(Y_valid1, y_pred_valid)
f1 = metrics.f1_score(Y_valid1, y_pred_valid)

Evaluating the trained model againt the Validation set

In [ ]:
print("Precision: ", precision)
print("Accuracy: ", accuracy)
print("F1 Score: ", f1)

Tuning the parameters

In [ ]:
clf = IForest(contamination=0.30)  # Changing contamination level  
clf.fit(train_Data_Set1)

In [ ]:
train_decision_scores = clf1.decision_scores_
train_decision_scores_sorted=sorted(train_decision_scores, reverse=True)
con_index = round(len(X_train1) * clf1.contamination)
threshold = train_decision_scores_sorted[con_index-1]

In [ ]:
valid_decision_scores = clf1.decision_function(valid_Data_Set1)  # outlier scores
valid_decision_scores = pd.Series(valid_decision_scores)

In [ ]:
valid_SrcIP = np.load('valid_set1_IPs.npy',allow_pickle=True)

In [ ]:
valid_outliers = []
y_pred_valid = []
for score in range(0,len(valid_decision_scores)):
  if valid_decision_scores[score] > threshold:
    reg = (valid_SrcIP[score], valid_decision_scores[score])
    valid_outliers.append(reg)
    y_pred_valid.append(1.0)
  else:
    y_pred_valid.append(0.0)

In [ ]:
precision = metrics.precision_score(valid_Set1_Labels, y_pred_valid)
accuracy = metrics.accuracy_score(valid_Set1_Labels, y_pred_valid)
f1 = metrics.f1_score(valid_Set1_Labels, y_pred_valid)

Evaluating the trained model against the validation set

In [ ]:
print("Precision: ", precision)
print("Accuracy: ", accuracy)
print("F1 Score: ", f1)

Getting predictions for the Test set

In [ ]:
test_decision_scores = clf1.decision_function(test_Data_Set1)  # outlier scores
test_decision_scores = pd.Series(test_decision_scores)

In [ ]:
test_SrcIP = np.load('test_set1_IPs.npy',allow_pickle=True)

Getting IP addresses of the predicted outliers

In [ ]:
outliers = []
y_pred_test = []
for score in range(0,len(test_decision_scores)):
  if test_decision_scores[score] > threshold:
    reg = (test_SrcIP[score], test_decision_scores[score])
    outliers.append(reg)
    y_pred_test.append(1.0)
  else:
    y_pred_test.append(0.0)

In [ ]:
ips = []
for i in outliers:
  ips.append(i[0])
ips = list(set(ips))

In [ ]:
cols = pd.DataFrame(columns=['Date_Flow_Start', 'Duration','Protocol','Src_IP','Src_Port','Direction','Dst_IP','Dst_Port','State','Source_Service','Dest_Service','Total_Packets','BiDirection_Bytes','SrcToDst_Bytes'])

In [ ]:
for each in ips:
  df = Test_Data[Test_Data.Src_IP == each]
  cols = cols.append(df)

In [ ]:
cols.drop(labels=['Direction', 'State', 'Source_Service',	'Dest_Service', 'Total_Packets', 'BiDirection_Bytes',	'SrcToDst_Bytes'], inplace=True, axis=1)

In [ ]:
cols.to_csv('Set1_iForest_1037144.csv', index = True)

Repeating the same processes for 2nd set

In [ ]:
clf = IForest() 
clf.fit(train_Data_Set2)

In [ ]:
decision_scores = clf1.decision_scores_
decision_scores_sorted=sorted(decision_scores, reverse=True)
con_index = round(len(train_Data_Set1) * clf.contamination)
threshold = decision_scores_sorted[con_index-1]

In [ ]:
valid_decision_scores = clf1.decision_function(valid_Data_Set2)  # outlier scores
valid_decision_scores = pd.Series(valid_decision_scores)

In [ ]:
valid_SrcIP = np.load('valid_set2_IPs.npy',allow_pickle=True)

In [ ]:
outliers = []
y_pred_valid = []
for score in range(0,len(valid_decision_scores)):
  if valid_decision_scores[score] > threshold:
    reg = (valid_SrcIP[score], valid_decision_scores[score])
    outliers.append(reg)
    y_pred_valid.append(1.0)
  else:
    y_pred_valid.append(0.0)

Evaluating the model

In [ ]:
precision = metrics.precision_score(valid_Set1_Labels, y_pred_valid) 
accuracy = metrics.accuracy_score(valid_Set1_Labels, y_pred_valid)
f1 = metrics.f1_score(valid_Set1_Labels, y_pred_valid)

In [ ]:
print("Precision: ", precision)
print("Accuracy: ", accuracy)
print("F1 Score: ", f1)

Tuning the model

In [ ]:
clf = IForest(contamination=0.25) 
clf.fit(train_Data_Set2)

In [ ]:
decision_scores = clf1.decision_scores_
decision_scores_sorted=sorted(decision_scores, reverse=True)
con_index = round(len(train_Data_Set2) * clf.contamination)
threshold = decision_scores_sorted[con_index-1]

In [ ]:
valid_decision_scores = clf.decision_function(valid_Data_Set2)  
valid_decision_scores = pd.Series(valid_decision_scores)

In [ ]:
valid_SrcIP = np.load('valid_set2_IPs.npy',allow_pickle=True)

In [ ]:
outliers = []
y_pred_valid = []
for score in range(0,len(valid_decision_scores)):
  if valid_decision_scores[score] > threshold:
    reg = (valid_SrcIP[score], valid_decision_scores[score])
    outliers.append(reg)
    y_pred_valid.append(1.0)
  else:
    y_pred_valid.append(0.0)

In [ ]:
precision = metrics.precision_score(valid_Set1_Labels, y_pred_valid)
accuracy = metrics.accuracy_score(valid_Set1_Labels, y_pred_valid)
f1 = metrics.f1_score(valid_Set1_Labels, y_pred_valid)

Evaluating the tuned model

In [ ]:
print("Precision: ", precision)
print("Accuracy: ", accuracy)
print("F1 Score: ", f1)

Testing the model on the Test Data

In [ ]:
decision_scores = clf.decision_function(test_Data_Set2)  # outlier scores
decision_scores = pd.Series(decision_scores)

In [ ]:
test_SrcIP = np.load('test_set2_IPs.npy',allow_pickle=True)

In [ ]:
outliers = []
y_pred_test = []
for score in range(0,len(y_test_scores)):
  if y_test_scores[score] > threshold:
    reg = (test_SrcIP[score], y_test_scores[score])
    outliers.append(reg)
    y_pred_test.append(1.0)
  else:
    y_pred_test.append(0.0)

In [ ]:
ips = []
for each in outliers:
  ips.append(each[0])
ips = list(set(ips))

In [ ]:
cols = pd.DataFrame(columns=['Date_Flow_Start', 'Duration','Protocol','Src_IP','Src_Port','Direction','Dst_IP','Dst_Port','State','Source_Service','Dest_Service','Total_Packets','BiDirection_Bytes','SrcToDst_Bytes'])

In [ ]:
for each in ips:
  df = Test_Data[Test_Data.Src_IP == each]
  cols = cols.append(df)

In [ ]:
cols.drop(labels=['Direction', 'State', 'Source_Service',	'Dest_Service', 'Total_Packets', 'BiDirection_Bytes',	'SrcToDst_Bytes'], inplace=True, axis=1)

In [ ]:
cols.to_csv('Set2_iForest_1037144.csv', index = True)


Now on the 3rd Set

In [ ]:
clf = IForest()
clf.fit(train_Data_Set3)

In [ ]:
decision_scores = clf1.decision_scores_
decision_scores_sorted=sorted(decision_scores, reverse=True)
con_index = round(len(X_train3) * clf1.contamination)
threshold = decision_scores_sorted[con_index-1]

In [ ]:
valid_decision_scores = clf.decision_function(valid_Data_Set3)  # outlier scores
valid_decision_scores = pd.Series(valid_decision_scores)

In [ ]:
valid_SrcIP = np.load('valid_set2_IPs.npy',allow_pickle=True)

In [ ]:
outliers = []
y_pred_valid = []
for score in range(0,len(valid_decision_scores)):
  if valid_decision_scores[score] > threshold:
    reg = (valid_SrcIP[score], valid_decision_scores[score])
    outliers.append(reg)
    y_pred_valid.append(1.0)
  else:
    y_pred_valid.append(0.0)

In [ ]:
precision = metrics.precision_score(valid_Set1_Labels, y_pred_valid)
accuracy = metrics.accuracy_score(valid_Set1_Labels, y_pred_valid)
f1 = metrics.f1_score(valid_Set1_Labels, y_pred_valid)

In [ ]:
print("Precision: ", precision)
print("Accuracy: ", accuracy)
print("F1 Score: ", f1)

Tuning the model

In [ ]:
clf = IForest(contamination=0.20) 
clf.fit(train_Data_Set3)

In [ ]:
decision_scores = clf1.decision_scores_
decision_scores_sorted=sorted(decision_scores, reverse=True)
con_index = round(len(X_train3) * clf1.contamination)
threshold = decision_scores_sorted[con_index-1]

In [ ]:
valid_decision_scores = clf1.decision_function(valid_Data_Set3)  # outlier scores
valid_decision_scores = pd.Series(valid_decision_scores)

In [ ]:
valid_SrcIP = np.load('valid_set2_IPs.npy',allow_pickle=True)

In [ ]:
outliers = []
y_pred_valid = []
for score in range(0,len(y_valid_scores)):
  if y_valid_scores[score] > threshold:
    reg = (valid_SrcIP[score], y_valid_scores[score])
    outliers.append(reg)
    y_pred_valid.append(1.0)
  else:
    y_pred_valid.append(0.0)

In [ ]:
precision = metrics.precision_score(valid_Set1_Labels, y_pred_valid)
accuracy = metrics.accuracy_score(valid_Set1_Labels, y_pred_valid)
f1 = metrics.f1_score(valid_Set1_Labels, y_pred_valid)

In [ ]:
print("Precision: ", precision)
print("Accuracy: ", accuracy)
print("F1 Score: ", f1)

Testing the model now on the test set 3

In [ ]:
valid_decision_scores = clf1.decision_function(test_Data_Set3)  # outlier scores
valid_decision_scores = pd.Series(valid_decision_scores)

In [ ]:
test_SrcIP = np.load('test_set2_IPs.npy',allow_pickle=True)

In [ ]:
outliers = []
y_pred_test = []
for score in range(0,len(y_test_scores)):
  if y_test_scores[score] > threshold:
    reg = (test_SrcIP[score], y_test_scores[score])
    test_outliers.append(reg)
    y_pred_test.append(1.0)
  else:
    y_pred_test.append(0.0)

In [ ]:
ips = []
for each in outliers:
  ips.append(each[0])
ips = list(set(ips))

In [ ]:
cols = pd.DataFrame(columns=['Date_Flow_Start', 'Duration','Protocol','Src_IP','Src_Port','Direction','Dst_IP','Dst_Port','State','Source_Service','Dest_Service','Total_Packets','BiDirection_Bytes','SrcToDst_Bytes'])

In [ ]:
for i in ips:
  df = Test_Data[Test_Data.Src_IP == i]
  cols = cols.append(df)

In [ ]:
cols.drop(labels=['Direction', 'State', 'Source_Service',	'Dest_Service', 'Total_Packets', 'BiDirection_Bytes',	'SrcToDst_Bytes'], inplace=True, axis=1)

In [ ]:
cols.to_csv('Set3_iForest_1037144.csv', index = True)